In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")

new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE


In [32]:
def get_float(msg:str, failed=False):
    """
    :param msg: message to display
    :param failed: flag if user has failed the question requirments
    :return : float number
    """
    if failed:
        print("Please enter a number")
    resp = input(msg+ " ")
    if resp.replace('.','').isnumeric():
        return float(resp)
    else:
        return get_float(msg,failed=True)
    
def get_int(msg:str, failed=False):
    """
    :param msg: message to display
    :param failed: flag if user has failed the question requirments
    :return : float number
    """
    if failed:
        print("Please enter a number")
    resp = input(msg+ " ")
    if resp.isnumeric():
        return int(resp)
    else:
        return get_float(msg,failed=True)
def get_yes_no(msg:str, failed=False):
    """
    Asks user for Y/N response
    :return : True for Yes, False for No
    """
    if failed:
        print("Please enter y or n as you answer")
    resp = input(msg+ " ")
    if resp.lower()=='n':
        return False
    elif resp.lower()=='y':
        return True
    else:
        return get_yes_no(msg,failed=True)

In [30]:
get_float("hey")

hey 5.25


5.25

## Step 1. Load the CE System with a Config File

Select one of the predesigned configurations (TE300, Eclipse Ti, Simulation). See an example config under "/config/test-system.cfg". 

Change the variable CONFIG_PATH to the absolute path where the config file resides. 

Be sure to make the string a raw string by prepending *'r'*. **Example:** r"C:\path\to\config\"*

In [2]:
CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()
ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x0000028672E832C8>, 'xy_stage': <L2.XYControl.PriorXY object at 0x0000028672DF0688>, 'objective': <L2.ZControl.PriorZ object at 0x0000028672E83388>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x0000028672E83408>, 'inlet_z': <L2.ZControl.KinesisZ object at 0x0000028672E83308>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x0000028672E83448>, 'camera': None, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x0000028672E83948>}


# Step 2. Load the Template File

The template file tells the program where to move the XY stage for each well as well as how high to raise the capillary inlet during a move. Change the variable TEMPLATE_PATH to the absolute path where the template.txt file resides. 

In [4]:
TEMPLATE_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\80WellTemplate.txt"
from L4 import AutomatedControl
reload(AutomatedControl)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)

# Step 3. Calibrate the Capillary Inlet

The method for calibration depends on the z-stage used to raise the capillary inlet. Select the correct route for the hardware employed. 

## Thorlabs Kinesis Z Stage

### Step 3.1 Lower the Capillary (thorlabs)
Move the capillary to a safe location where it will not be crushed if the stage lowers too far. Using the controls on the Z-stage itself, lower the capillary till as far as it will go. 


In [8]:
resp = get_yes_no("Have you lowered the Kinesis Z stage to its lowest position? (y/n):  ")
if not resp:
    print("Please make sure the Kinesis stage is lowered to its lowest position before continuing")
else:
    print("Proceed to the next cell.")
    

Have you lowered the Kinesis Z stage to its lowest position? (y/n):  y
Proceed to the next cell.


### Step 3.2 Home the Capillary (thorlabs)

Thorlabs needs to register the home position. In the next cell we will call the go_home command to home the capillary. 

In [9]:
ce_system.inlet_z.homing()

### Step 3.3 Adjust Capillary Position (all stages)

Adjust the capillary position by sliding the detection apparatus up or down the optical support post. The capillary should be level with the bottom of the template for single-cell assays, or the top-of the template for well-based assays. 

*Aligning to the top of the template is a minor safety feature to prevent the user from sending the capillary to Zero position unexpetedly and crashing the capillary against the bottom of the template. However, capillary needs to be able to reach at least the glass slide of the cell reservoir for single-cell assays*


In [10]:
resp = get_yes_no("Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):  ")
if not resp:
    print("Please make sure the capillary is moved to the correct height with respect to the template before continuing")
else:
    print("Proceed to the next cell")

Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):  y
Proceed to the next cell


### Step 3.4 Raise the capillary to a safe height

The next cell will raise the capillary height to a location where it ***should*** be safe for any XY movements. Verify the capillary will not crash at this new height. 

In [31]:
target_z = get_float("Please enter a safe height to move the stage to: ")
# Move the capillary up and wait for it to stop
ce_system.inlet_z.set_z(target_z)
ce_system.inlet_z.wait_for_target(target_z)

resp = get_yes_no("Is the capillary at a safe location?: ")
if resp:
    print("Continue to next cell")
else:
    print("Please raise the capillary to an appropriate height")

Please enter a safe height to move the stage to:  25
Is the capillary at a safe location?y
Continue to next cell


# Step 4. Home the XY Stage

This next step provides the calibration for the XY axis for the XY stage and capillary inlet. 
